In [236]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import datetime
from geopy import distance
import statsmodels.api as sm
from scipy import stats
from math import *

In [237]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")
Rider = pd.read_csv("Riders.csv")
sp = pd.read_csv("SampleSubmission.csv")

In [238]:
# ORS
ors_test = pd.read_csv("ors_test.csv")
ors_train = pd.read_csv("ors_train.csv")

In [239]:
print(ors_test.shape)
print(ors_train.shape)

(7068, 2)
(21201, 2)


In [240]:
Train.columns

Index(['Order No', 'User Id', 'Vehicle Type', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Arrival at Destination - Day of Month',
       'Arrival at Destination - Weekday (Mo = 1)',
       'Arrival at Destination - Time', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Rider Id',
       'Time from Pickup to Arrival'],
      dtype='object')

In [241]:
Train["distance_ors"] = ors_train["distance"]/1000  # en mètre / sec
Test["distance_ors"] = ors_test["distance"]/1000
Train["duration_ors"] = ors_train["duration"]  # sec
Test["duration_ors"] = ors_test["duration"]
Train = pd.merge(Train, Rider, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)
Test = pd.merge(Test, Rider, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)

In [148]:
col_x = ["Platform Type","Personal or Business","Pickup Long","Pickup Lat","Destination Long","Destination Lat","Distance (KM)","Temperature","Precipitation in millimeters","Pickup - Weekday (Mo = 1)","Pickup - Time","No_Of_Orders","Age","Average_Rating","duration_ors","distance_ors"]
col_x = sorted(col_x)
col_y = ["Time from Pickup to Arrival"]
Train = Train[col_x+col_y]
Test = Test[col_x]

In [242]:
def convert_time24(nom,col,data=Train):
 tab = []
 for i in col:
   s = i.split()
   terminaison = s[1]
   if terminaison == "PM":
     time = s[0].split(":")
     time = (int(time[0]) + 12) * 3600 + int(time[1]) * 60 + int(time[2])
     tab.append(time)
   else:
     time = s[0].split(":")
     time = (int(time[0]) + 12) * 3600 + int(time[1]) * 60 + int(time[2])
     tab.append(time)
 data[nom] = tab
 return True
 # Train
#convert_time24("Placement - Time",Train["Placement - Time"])
#convert_time24("Confirmation - Time",Train["Confirmation - Time"])
#convert_time24("Pickup - Time",Train["Pickup - Time"])
#Test
#convert_time24("Placement - Time",Test["Placement - Time"],Test)
#convert_time24("Confirmation - Time",Test["Confirmation - Time"],Test)
#convert_time24("Pickup - Time",Test["Pickup - Time"],Test)

In [150]:
Train['Temperature'] = Train['Temperature'].fillna(Train['Temperature'].mean())
Test['Temperature'] = Test['Temperature'].fillna(Test['Temperature'].mean())
Train['Precipitation in millimeters'] = Train['Precipitation in millimeters'].fillna(0)
Test['Precipitation in millimeters'] = Test['Precipitation in millimeters'].fillna(0)

In [243]:
def heure(data):
 time=[]
 data = pd.to_datetime(data)
 for i in data:
     i = i.time()
     ti = (str(i).split(':')[0])
     time.append(int(ti))
 time
 data = time
 return data

In [152]:
print(heure(Train ['Pickup - Time']))

[10, 11, 12, 9, 10, 15, 9, 14, 12, 17, 10, 16, 15, 9, 10, 13, 12, 12, 11, 14, 10, 11, 19, 10, 15, 12, 15, 12, 14, 8, 15, 9, 16, 16, 14, 17, 13, 13, 17, 8, 11, 14, 17, 13, 14, 11, 14, 15, 12, 11, 11, 10, 17, 9, 13, 15, 13, 12, 10, 11, 11, 11, 11, 10, 10, 17, 13, 10, 15, 10, 17, 11, 13, 11, 10, 13, 16, 10, 15, 11, 11, 13, 14, 12, 10, 11, 13, 12, 12, 12, 11, 15, 11, 14, 15, 15, 11, 15, 9, 20, 10, 16, 8, 10, 17, 10, 9, 15, 8, 12, 15, 14, 15, 11, 9, 11, 11, 12, 17, 16, 12, 19, 14, 10, 10, 13, 13, 15, 11, 13, 15, 13, 17, 13, 11, 12, 17, 10, 10, 14, 12, 17, 11, 12, 12, 12, 14, 11, 15, 10, 13, 15, 10, 16, 15, 17, 16, 17, 14, 12, 13, 12, 12, 16, 12, 16, 9, 13, 11, 16, 12, 9, 17, 13, 14, 11, 15, 15, 12, 17, 15, 13, 19, 14, 16, 14, 15, 14, 9, 10, 14, 10, 12, 13, 10, 9, 9, 12, 11, 12, 12, 16, 11, 9, 10, 10, 14, 12, 14, 8, 12, 9, 16, 14, 11, 12, 13, 15, 10, 14, 9, 15, 16, 12, 14, 14, 14, 12, 15, 15, 13, 14, 10, 11, 13, 15, 12, 14, 13, 13, 11, 15, 19, 10, 13, 10, 17, 11, 9, 11, 13, 15, 11, 16, 16, 1

In [153]:
print(Train ['Pickup - Time'])

27     10:27:30 AM
739    11:44:09 AM
851    12:53:03 PM
806     9:43:06 AM
159    10:05:23 AM
          ...     
712     4:20:08 PM
851    10:33:27 AM
642     5:50:52 PM
41      9:45:15 AM
801     2:41:55 PM
Name: Pickup - Time, Length: 21201, dtype: object


In [154]:
Train["Pickup - Time"] = heure(Train ['Pickup - Time'])
Test['Pickup - Time'] = heure(Test ['Pickup - Time'])

In [155]:
Train.dtypes

Age                               int64
Average_Rating                  float64
Destination Lat                 float64
Destination Long                float64
Distance (KM)                     int64
No_Of_Orders                      int64
Personal or Business             object
Pickup - Time                     int64
Pickup - Weekday (Mo = 1)         int64
Pickup Lat                      float64
Pickup Long                     float64
Platform Type                     int64
Precipitation in millimeters    float64
Temperature                     float64
distance_ors                    float64
duration_ors                    float64
Time from Pickup to Arrival       int64
dtype: object

In [244]:
def dummy(dataset,columns):
 for c in columns:
   dum = pd.get_dummies(dataset[c],prefix=c,prefix_sep="_")
   dataset = pd.concat([dataset,dum], axis=1)
   del dataset[c]
 return dataset

In [235]:
print(Train.shape)
print(Test.shape)

(21201, 35)
(7068, 31)


In [101]:
dummies_col = ["Platform Type","Personal or Business","Pickup - Weekday (Mo = 1)"]
Train = dummy(Train,dummies_col)
Test =dummy(Test,dummies_col)

In [102]:
print(Train.shape)
print(Test.shape)

(21201, 27)
(7068, 26)


In [77]:
Train.describe()

,Age,Average_Rating,Destination Lat,Destination Long,Distance (KM),No_Of_Orders,Pickup - Time,Pickup Lat,Pickup Long,Precipitation in millimeters,...,Platform Type_4,Personal or Business_Business,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_1,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
count,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,...,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000
mean,984.742842,13.882520,-1.282581,36.811220,9.506533,1692.423706,12.873214,-1.281470,36.811264,0.205839,...,0.000943,0.819961,0.180039,0.178671,0.186736,0.180322,0.199472,0.188340,0.057686,0.008773
std,646.652835,0.916071,0.034824,0.044721,5.668963,1574.308302,2.551666,0.030507,0.037473,3.029194,...,0.030700,0.384229,0.384229,0.383085,0.389709,0.384464,0.399613,0.390993,0.233154,0.093256
min,96.000000,0.000000,-1.430298,36.606594,1.000000,2.000000,0.000000,-1.438302,36.653621,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,495.000000,13.600000,-1.301201,36.785661,5.000000,557.000000,11.000000,-1.300921,36.784605,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,872.000000,14.000000,-1.284382,36.808002,8.000000,1212.000000,13.000000,-1.279395,36.807040,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1236.000000,14.300000,-1.261177,36.829477,13.000000,2311.000000,15.000000,-1.257147,36.829741,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3764.000000,15.200000,-1.030225,37.016779,49.000000,9756.000000,23.000000,-1.147170,36.991046,99.100000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [78]:
Train.corr()

,Age,Average_Rating,Destination Lat,Destination Long,Distance (KM),No_Of_Orders,Pickup - Time,Pickup Lat,Pickup Long,Precipitation in millimeters,...,Platform Type_4,Personal or Business_Business,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_1,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
Age,1.000000,-0.014348,0.009552,-0.018235,0.036239,0.669036,-0.001511,0.002481,-0.001656,0.012079,...,0.010488,-0.071117,0.071117,0.009234,-0.007640,0.015558,0.004047,-0.005540,-0.027755,0.005128
Average_Rating,-0.014348,1.000000,0.030488,0.002017,-0.008200,0.051375,-0.003933,0.043610,-0.001885,-0.000432,...,0.000083,-0.016044,0.016044,-0.005092,0.009897,0.001718,0.010002,-0.014144,-0.002103,-0.005824
Destination Lat,0.009552,0.030488,1.000000,0.046767,-0.079366,-0.001135,0.018985,0.114975,0.014992,-0.001703,...,0.010052,0.006672,-0.006672,0.007048,0.007509,-0.005982,-0.006686,-0.011362,0.008276,0.019927
Destination Long,-0.018235,0.002017,0.046767,1.000000,0.135397,-0.013809,-0.002764,0.051399,0.082803,0.006729,...,-0.003055,0.063084,-0.063084,0.006936,-0.006776,0.002683,0.002545,-0.012797,0.006439,0.015414
Distance (KM),0.036239,-0.008200,-0.079366,0.135397,1.000000,0.037186,-0.036373,-0.124338,0.178537,-0.001761,...,0.020021,0.007177,-0.007177,0.008455,-0.006477,0.004816,-0.015515,-0.005056,0.011724,0.030853
No_Of_Orders,0.669036,0.051375,-0.001135,-0.013809,0.037186,1.000000,-0.007390,-0.018190,0.011274,0.012714,...,0.001239,0.029981,-0.029981,0.010044,0.005400,0.013627,-0.000691,-0.011322,-0.029803,0.004933
Pickup - Time,-0.001511,-0.003933,0.018985,-0.002764,-0.036373,-0.007390,1.000000,0.039430,-0.011040,-0.078653,...,0.000323,-0.017269,0.017269,0.001991,-0.003418,-0.004342,0.007271,0.022895,-0.050421,0.022912
Pickup Lat,0.002481,0.043610,0.114975,0.051399,-0.124338,-0.018190,0.039430,1.000000,-0.147629,-0.005395,...,-0.031689,0.022352,-0.022352,-0.002727,0.004828,-0.009185,0.009309,0.007839,-0.021099,0.008891
Pickup Long,-0.001656,-0.001885,0.014992,0.082803,0.178537,0.011274,-0.011040,-0.147629,1.000000,-0.007141,...,0.013332,0.036729,-0.036729,-0.000427,-0.009653,0.003624,-0.008813,-0.000373,0.033362,-0.016928
Precipitation in millimeters,0.012079,-0.000432,-0.001703,0.006729,-0.001761,0.012714,-0.078653,-0.005395,-0.007141,1.000000,...,-0.001429,-0.008823,0.008823,-0.024178,0.077253,-0.015736,-0.015423,-0.013167,-0.012732,-0.005508


In [79]:
y_train = Train['Time from Pickup to Arrival']
c = list(set(Train.columns) - {"Time from Pickup to Arrival"})
X_train = Train[sorted(c)]
X_test = Test[sorted(c)]

# BON MODEL DE L'INSTANT with precipitation, average_of_rating, duration_ors & distance_ors


In [80]:
#!pip install lightgbm

In [192]:
from lightgbm import LGBMRegressor

In [193]:
lgbm = LGBMRegressor(
    num_leaves=65,
    max_depth=5,
    learning_rate=0.06,
    n_estimators=500
)



In [83]:
lgbm.fit(X_train,y_train)
print("score:",lgbm.score(X_train,y_train))

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

score: 0.5801405386153686


/home/moussa/snap/jupyter/common/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.350
Model:                                     OLS   Adj. R-squared:                  0.349
Method:                          Least Squares   F-statistic:                     495.8
Date:                         Sat, 16 Nov 2019   Prob (F-statistic):               0.00
Time:                                 13:50:52   Log-Likelihood:            -1.7170e+05
No. Observations:                        21201   AIC:                         3.434e+05
Df Residuals:                            21177   BIC:                         3.436e+05
Df Model:                                   23                                         
Covariance Type:                     nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

In [84]:
y_pred_ = lgbm.predict(X_test)

In [85]:
sp["Time from Pickup to Arrival"] = y_pred_
sp.to_csv("zindi_30000.csv",index=False) # 727.863210902153

# essai

In [245]:
Train["vitesse_ors"] =Train["distance_ors"] / Train["duration_ors"] # en mètre / sec
Test["vitesse_ors"] = Test["distance_ors"]/Test["duration_ors"] 
#Train["vitesse"] =Train["Distance (KM)"] / Train["duration_ors"] # en mètre / sec
#Test["vitesse"] = Test["Distance (KM)"]/Test["duration_ors"] 


In [247]:
Train.describe()

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Arrival at Destination - Day of Month,...,Destination Lat,Destination Long,Time from Pickup to Arrival,distance_ors,duration_ors,No_Of_Orders,Age,Average_Rating,No_of_Ratings,vitesse_ors
count,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,...,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000
mean,2.752182,15.653696,3.240083,15.653837,3.240225,15.653837,3.240225,15.653837,3.240225,15.653837,...,-1.282581,36.811220,1556.920947,9.242227,1916.466006,1692.423706,984.742842,13.882520,341.067119,0.004839
std,0.625178,8.798916,1.567295,8.798886,1.567228,8.798886,1.567228,8.798886,1.567228,8.798886,...,0.034824,0.044721,987.270788,5.940761,1253.986171,1574.308302,646.652835,0.916071,402.867746,0.000198
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.430298,36.606594,1.000000,0.102700,23.600000,2.000000,96.000000,0.000000,0.000000,0.000660
25%,3.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,...,-1.301201,36.785661,882.000000,4.715500,970.100000,557.000000,495.000000,13.600000,61.000000,0.004780
50%,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,...,-1.284382,36.808002,1369.000000,7.919500,1635.700000,1212.000000,872.000000,14.000000,161.000000,0.004902
75%,3.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,...,-1.261177,36.829477,2040.000000,12.448900,2556.300000,2311.000000,1236.000000,14.300000,495.000000,0.004958
max,4.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,...,-1.030225,37.016779,7883.000000,48.331900,9991.300000,9756.000000,3764.000000,15.200000,2298.000000,0.013264


In [248]:
col_x = ["Platform Type","Personal or Business","Pickup Long","Pickup Lat","Destination Long","Destination Lat","Distance (KM)","Pickup - Weekday (Mo = 1)","Pickup - Time","No_Of_Orders","Age","Average_Rating","duration_ors","distance_ors","vitesse_ors"]
col_x = sorted(col_x)
col_y = ["Time from Pickup to Arrival"]
Train = Train[col_x+col_y]
Test = Test[col_x]

In [249]:
def heure(data):
 time=[]
 data = pd.to_datetime(data)
 for i in data:
     i = i.time()
     ti = (str(i).split(':')[0])
     time.append(int(ti))
 time
 data = time
 return data

In [250]:
Train["Pickup - Time"] = heure(Train ['Pickup - Time'])
Test['Pickup - Time'] = heure(Test ['Pickup - Time'])

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [251]:
Train.dtypes

Age                              int64
Average_Rating                 float64
Destination Lat                float64
Destination Long               float64
Distance (KM)                    int64
No_Of_Orders                     int64
Personal or Business            object
Pickup - Time                    int64
Pickup - Weekday (Mo = 1)        int64
Pickup Lat                     float64
Pickup Long                    float64
Platform Type                    int64
distance_ors                   float64
duration_ors                   float64
vitesse_ors                    float64
Time from Pickup to Arrival      int64
dtype: object

In [252]:
def dummy(dataset,columns):
 for c in columns:
   dum = pd.get_dummies(dataset[c],prefix=c,prefix_sep="_")
   dataset = pd.concat([dataset,dum], axis=1)
   del dataset[c]
 return dataset

In [253]:
dummies_col = ["Platform Type","Personal or Business","Pickup - Weekday (Mo = 1)"]
Train = dummy(Train,dummies_col)
Test =dummy(Test,dummies_col)

In [254]:
y_train = Train['Time from Pickup to Arrival']
c = list(set(Train.columns) - {"Time from Pickup to Arrival"})
X_train = Train[sorted(c)]
X_test = Test[sorted(c)]

In [255]:
lgbm.fit(X_train,y_train)
print("score:",lgbm.score(X_train,y_train))

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

score: 0.5802547935774355
                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.350
Model:                                     OLS   Adj. R-squared:                  0.349
Method:                          Least Squares   F-statistic:                     518.6
Date:                         Sun, 17 Nov 2019   Prob (F-statistic):               0.00
Time:                                 00:48:59   Log-Likelihood:            -1.7169e+05
No. Observations:                        21201   AIC:                         3.434e+05
Df Residuals:                            21178   BIC:                         3.436e+05
Df Model:                                   22                                         
Covariance Type:                     nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.9

/home/moussa/snap/jupyter/common/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [256]:
y_pred_ = lgbm.predict(X_test)

In [257]:
len(y_pred_)

7068

In [199]:
sp["Time from Pickup to Arrival"] = y_pred_
sp.to_csv("zindi_30000_vitesse_ors.csv",index=False) 

# essaie2

In [210]:
col_x = ["Platform Type","Personal or Business","Distance (KM)","Pickup - Weekday (Mo = 1)","Pickup - Time","No_Of_Orders","Age","Average_Rating","duration_ors","distance_ors"]
col_x = sorted(col_x)
col_y = ["Time from Pickup to Arrival"]
Train = Train[col_x+col_y]
Test = Test[col_x]

In [211]:
Train["Pickup - Time"] = heure(Train ['Pickup - Time'])
Test['Pickup - Time'] = heure(Test ['Pickup - Time'])

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [215]:
Test.describe()

,Age,Average_Rating,Distance (KM),No_Of_Orders,Pickup - Time,distance_ors,duration_ors,Platform Type_1,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Business,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_1,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
count,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000
mean,998.400113,13.883178,9.458970,1717.032258,12.837295,9.197450,1905.711573,0.100594,0.047538,0.851019,0.000849,0.817770,0.182230,0.181098,0.184352,0.177844,0.196095,0.190860,0.060413,0.009338
std,672.195000,0.846333,5.575689,1628.842795,2.550714,5.836542,1228.364291,0.300812,0.212802,0.356095,0.029126,0.386061,0.386061,0.385127,0.387798,0.382408,0.397069,0.393007,0.238268,0.096187
min,99.000000,0.000000,1.000000,3.000000,7.000000,0.153600,30.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,494.000000,13.600000,5.000000,534.000000,11.000000,4.822575,999.700000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,872.000000,14.000000,8.000000,1209.000000,13.000000,7.930000,1634.650000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1239.000000,14.300000,13.000000,2311.000000,15.000000,12.381800,2537.300000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3764.000000,15.200000,47.000000,9756.000000,23.000000,46.515300,9636.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [216]:
lgbm.fit(X_train,y_train)
print("score:",lgbm.score(X_train,y_train))

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

score: 0.5196599539436021
                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.348
Model:                                     OLS   Adj. R-squared:                  0.348
Method:                          Least Squares   F-statistic:                     665.3
Date:                         Sat, 16 Nov 2019   Prob (F-statistic):               0.00
Time:                                 14:26:43   Log-Likelihood:            -1.7173e+05
No. Observations:                        21201   AIC:                         3.435e+05
Df Residuals:                            21183   BIC:                         3.436e+05
Df Model:                                   17                                         
Covariance Type:                     nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.9

/home/moussa/snap/jupyter/common/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [217]:
y_pred_ = lgbm.predict(X_test)

In [219]:
sp["Time from Pickup to Arrival"] = y_pred_
sp.to_csv("zindi_300003.csv",index=False) 